In [1]:
using ITensors
#import KrylovKit 
#using KrylovKit
include("TamaTimeEvoMPS/src/TimeEvoMPS.jl")
using .TimeEvoMPS

In [2]:
using DelimitedFiles
coups = readdlm("Chain_mappings/WSCP_MC_T0_coups.dat");
freqs = readdlm("Chain_mappings/WSCP_MC_T0_freqs.dat");
#system energy gap
eps = 138.

138.0

In [3]:
sys = siteinds("S=1/2",1);
NChain = 86;
env = siteinds("Boson", dim=6, NChain);
NN = NChain + 1;
sysenv = vcat(sys,env);

In [4]:
stateSys = ["Up"];
stateEnv = ["0" for n=1:NChain];
stateSE = vcat(stateSys,stateEnv);
psi0 = productMPS(sysenv,stateSE);

In [5]:
length(sysenv)

87

We define a trivial Hamiltonian to use as to set the initial bond dimension of the MPS to the desired size (mindim)

In [6]:
# #Lavoriamo qui
# ampo = OpSum()
# #system Hamiltonian

# for j=1:length(sysenv)
#    ampo += 0.001, "Id",j
# end
# H = MPO(ampo,sysenv); 
wantedDim = 8
psiExt = copy(psi0);
for n in 1:NN-1
   a = commonind(psiExt[n],psiExt[n+1])
   tagsa = tags(a)
   add_indx = Index(wantedDim, tags= tagsa)
   psiExt[n]=psiExt[n]*delta(a,add_indx)
   psiExt[n+1]=psiExt[n+1]*delta(a,add_indx)
end

In [7]:
# appoPsi = copy(psi0);
# tdvp!(appoPsi,H,0.00001,0.00001,mindim = 8, maxdim=20, progress=true,cutoff=10e-20)

In [8]:
# dot(psi0,appoPsi)
dot(psi0,psiExt)

1.0

In [9]:
psi0=copy(psiExt);

In [10]:
#asymptotic frequency
MC_N = 6;
NP_Chain = NN - MC_N;
OO = 500.;
alphas_MC = readdlm("MC_Pars/alphas_6.dat")
betas_MC = readdlm("MC_Pars/betas_6.dat")
coups_MC = readdlm("MC_Pars/coupls_6.dat")
gammas = OO * alphas_MC[:,1];
eff_freq = [OO + 1im * g for g in gammas];
eff_gs = OO * betas_MC[:,2];
eff_coup = OO/2* (coups_MC[:,1]+ 1im *coups_MC[:,2]);

#Definition of the TEDOPA+MC Hamiltonian (MPO form)
#Lavoriamo qui
ampo = OpSum()
#system Hamiltonian
ampo += eps,"Sz",1;
#system-env interaction
#!Sx = 0.5 σx
ampo += 2*coups[1],"Sx",1,"Adag",2
ampo += 2*coups[1],"Sx",1,"A",2

#Primary chain local Hamiltonians
for j=2:NP_Chain
   ampo += freqs[j-1],"N",j
end

for j=2:NP_Chain-1
   ampo += coups[j],"A",j,"Adag",j+1
   ampo += coups[j],"Adag",j,"A",j+1
end
#################################

#Markovian closure Hamiltonian
for j=1:MC_N
   ampo += eff_freq[j],"N",NP_Chain+j
end

for j=1:MC_N-1
   ampo += eff_gs[j],"A",NP_Chain+j,"Adag",NP_Chain+j+1
   ampo += eff_gs[j],"Adag",NP_Chain+j,"A",NP_Chain+j+1
end

#################################

#Primary chain - MC interaction
for j=1:MC_N
   ampo += eff_coup[j],"A",NP_Chain,"Adag",NP_Chain+j
   ampo += conj(eff_coup[j]),"Adag",NP_Chain,"A",NP_Chain+j
end


H = MPO(ampo,sysenv); 

#Set MPO bond dimensions
[dim(linkind(H,j)) for j in 1:length(H)-1];

In [11]:
NP_Chain

81

In [12]:
vobs = []
# push!(vobs,opPos("Sx",1))
# push!(vobs,opPos("Sz",1))
# nchain = collect(10:10:10)
# for i in nchain 
#    push!(vobs,opPos("N",i+1))
# end

Any[]

In [13]:
psi=deepcopy(psi0)
timestep = 0.0001
tmax = 0.001
cbT = LocalMeasurementCallbackTama(vobs,sysenv,10*timestep)

LocalMeasurementCallback
Operators: opPos[]
No measurements performed


In [14]:
norm(psi)

1.0

In [15]:
tdvp1!(psi,H,timestep,tmax, ishermitian = false, issymmetric = false,  normalize = false, callback=cbT, progress=true, io_file="critico.dat",store_psi0 = true,io_ranks = "ranghi.dat",io_times = "times.dat")

Evolving state...  20%|██████▋                          |  ETA: 0:01:36
  t:             0.0002


  dt step time:  2.829





Evolving state...  30%|█████████▉                       |  ETA: 0:01:04
  t:             0.00030000000000000003
  dt step time:  2.875





Evolving state...  40%|█████████████▎                   |  ETA: 0:00:45
  t:             0.0004
  dt step time:  2.847





Evolving state...  50%|████████████████▌                |  ETA: 0:00:33
  t:             0.0005
  dt step time:  2.975





Evolving state...  60%|███████████████████▊             |  ETA: 0:00:24
  t:             0.0006000000000000001
  dt step time:  3.447





Evolving state...  70%|███████████████████████▏         |  ETA: 0:00:17
  t:             0.0007
  dt step time:  3.408





Evolving state...  80%|██████████████████████████▍      |  ETA: 0:00:11
  t:             0.0008
  dt step time:  3.148





Evolving state...  90%|█████████████████████████████▊   |  ETA: 0:00:05
  t:             0.0009000000000000001
  dt step time:  3.152





Evolving state... 100%|█████████████████████████████████| Time: 0:00:50
  t:             0.001
  dt step time:  3.574
  Max bond-dim:  8


In [16]:
dot(psi0,psi)

0.9993521446997559 + 1.2570286704335398e-5im

## Plotting

In [ ]:
using PyPlot
ts = measurement_ts(cbT)
for o in ["Sx_1","Sz_1","N_10","N_20"]
    S5 = getindex.(measurements(cbT)["$o"],1)
    plot(ts,S5,"-o",label="$o")
end
legend()
xlabel("t")

In [ ]:
psi[10]

In [ ]:
typeof(io)

In [ ]:
using Printf

In [ ]:
io = open("prova_sampa.dat","a")
@printf(io,"%20s%20s%20s\n","tempo","ciao","mondo")
@printf(io,"%20f%20f%20f\n",10,20,30)
close(io)